## Sofiia Druchyna
## CMPS 3160: Intro to Data Science Project
## Cyberattacks and Threats

In [2]:
!ls

FullAttackList.csv  FullThreatActors.csv  sample_data


In [3]:
import pandas as pd

df_threats = pd.read_csv('./FullThreatActors.csv')
display(df_threats)

,Name,Country,Actor Type,Established,Last Seen
0,$2a$45,Undetermined,Criminal,NaN,2018-03-09
1,'desserped',Undetermined,Criminal,NaN,2023-10-11
2,0mega,Undetermined,Criminal,NaN,2023-02-20
3,0x0D1337,Undetermined,Undetermined,NaN,2015-09-30
4,0x2Taylor,Undetermined,Hacktivist,NaN,2016-10-13
...,...,...,...,...,...
95,Angel_Of_Truth,Canada,Hacktivist,NaN,2015-04-21
96,AnibaLeaks,Undetermined,Criminal,NaN,2021-09-30
97,Anka Neferler Timi,France,Hacktivist,NaN,2020-12-22
98,AnoaGhost,Undetermined,Undetermined,NaN,2018-04-17


In [4]:
df_attack = pd.read_csv('./FullAttackList.csv')
df_attack

,Date,Location,Victim,Industry,Actor Location,Actor,Motive,Type,Sub-Type
0,2023-10-31,United States of America,Smoothie King,Accommodation and Food Services,Undetermined,Undetermined,Financial,Exploitive,Undetermined
1,2023-10-31,Italy,Avangate,Other Services (except Public Administration),Undetermined,Alpha Team,Financial,Mixed,Data Attack
2,2023-10-30,United States of America,Dallas County,Public Administration,Undetermined,PLAY,Financial,Mixed,Data Attack
3,2023-10-30,Russian Federation,National Payment Card System (NSPK),Finance and Insurance,Ukraine,DumpForums and Ukrainian Cyber Alliance,Protest,Disruptive,Message Manipulation
4,2023-10-30,Germany,Südwestfalen IT,"Professional, Scientific, and Technical Services",Undetermined,Undetermined,Financial,Mixed,Data Attack
...,...,...,...,...,...,...,...,...,...
295,2023-09-12,United States of America,East Jackson Community Schools,Educational Services,Undetermined,Undetermined,Undetermined,Undetermined,Undetermined
296,2023-09-12,Colombia,IFX Networks Colombia,Information,Undetermined,Ransom House,Financial,Mixed,Data Attack
297,2023-09-12,Italy,SDA,Transportation and Warehousing,Italy,ADHD,Protest,Mixed,Data Attack
298,2023-09-12,United States of America,Fauquier County Public Schools,Educational Services,Undetermined,LockBit 3.0,Financial,Mixed,Data Attack
